# **LAB THREE: SELECTIVE EXCITATION**

This lab covers...

>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
LAB_USER_NAME = 'Sharon'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [21]:
import panel as pn
pn.extension(raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import sys
import os
import numpy as np

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

User data directory: /home/data/Sharon


## 1. Projection Images
Lab two introduced how frequency and phase encoding can be used to sort an aquired signal into 2D space. Thus far, we have been collecting signal from the entire sample and separating out the components along the axes that the frequency and phase gradients were applied. This results in a 2D projection image where all the signal is summed in the third direction (shown in Figure 1). 

<center><img src="Images/projection.png" width="800"></center>
<center><figcaption style="width: 600px;">Figure 1: Setting the frequency and phase encoding in the x and z axes produces a projection imagine where the varying signal components in the y direction are summed into average values.  </figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 1.1: Generate a Projection Image**
> TO DO: Insert a window that is 2D RARE without slice selection options.
> 1. Insert the mystery sample (ilumr sample) at the correct depth.
> 2. Choose a the phase encoding and frequency encoding axes and start the experiment by pressing "Run". 
>
> **Question:** Can you read the text located in the centre of the phantom?
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

As you can see, projection images aren't always very useful because summing a large amount of signal in the object's third dimension makes it almost impossible to see details that are only present in specific parts of the sample. If we want to see detail in a certain portion of the object we need to be able to limit the amount of signal that is being summed in one direction. This is done through slice selection.

<center><img src="Images/slice.png" width="800"></center>
<center><figcaption style="width: 600px;">Figure 2: Applying a slice select gradient along y allows...  </figcaption></center>

## 2. Slice Selection
You are already familiar with the fact that applying a linear gradient along a certain axis causes the spins to preccess at different frequencies along that axis. So far we have been using a rectangular RF-pulse (known as a hard pulse) to excite these spins and flip them into the transverse plane. Since the rectangle pulse contains all of the frequencies, it excites all of the spins in the sample (shown in Fig.x).

In slice selection, we only want to excite the spins within the desired section of the sample. This means that we need to limit the frequencies that the RF-pulse contains. If we want a rectangular slice profile in the frequency domain, we will need to apply a sinc shaped RF-pulse (known as a soft pulse) in the time domain (shown in Fig.x). We can vary the thickness of these slices and they can be centered at different positions along the slice select gradient axis.  



TO DO: Explain why a perfect sinc can not be used and how different compromising are made like limiting nodes and using gaussian distributions. ilumr uses SLR optimisation algorithm to design its sinc-like soft pulses. 

<center><img src="Images/selective_excitation 3.0.png" width="1500"></center>
<center><figcaption style="width: 600px;">Figure 3: </figcaption></center>

## 3. Controlling Slice Thickness
The thickness of the slice ($\Delta$z) is controlled by two main factors, the frequency range selected by the soft pulse ($\Delta$F) and the slice select gradient ($G_{ss}$). This relationship is governed by the following equation:

$$\Delta z = \frac{\Delta F}{\gamma . G_{ss}} \tag{1}$$

where $\gamma$ is the gyromagnetic ratio. 

A soft pulse has three key parameters: the bandwidth, duration, and amplitude. 

TO DO: 
- explain each of these and the affect they have on the shape of the pulse. 
- explain how slice width is calculated using these parameters 


<center><img src="Images/soft_pulse_parameters.png" width="800"></center>
<center><figcaption style="width: 300px;">Figure x:  </figcaption></center>

### 3.1 Soft Pulses
TO DO: Talk about the different types of soft pulses (sinc, gaussian, optimisation algorithms). Outline the key focus of the section: changing the parameters of the RF-pulses changes the slice profile and resulting image quality. Breifly mention that there are limitations. 

#### 3.1.1 Sinc Pulse
TO DO: explain sinc pulses. The ideal slice profile is a perfect rectangle that only excites the spins in the limited volume of the sample (show this). Theoretically, this can be achieved using a sinc pulse (add equation). However physical limitations mean that the sinc pulse has to be altered.. duration, node limiting, apodization etc talk about why these need to happen. 

TO DO: Explain the parameters of the sinc pulse - duration, number of lobes, apodization, amplitude and hhow they relate to shape and bandwidth. Give enough info to make the following exercise make sense.

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.1.1: Visualising Sinc Pulse Design**
> TO DO: Insert a window that plots how the soft pulse and the frequency spectrum changes based on the bandwidth and width parameters. Allow students to change the duration, number of nodes and apodization and see how it changes the shape of the sinc FFT. Compare this shape to a ideal fourier transform of the sinc. 
> 1. Experiment with the three soft pulse parameters and observe how the shape of the RF-pulse and slice profile changes. 
> 2. Assuming the gradient is set to 80%, use the equations above to calculate a 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
from matipo.util.pulseshape import calc_soft_pulse
from matipo.util.plots import SharedXPlot, ComplexPlot
from scipy.fft import fft,fftfreq, fftshift, ifftshift
from bokeh.plotting import figure

width_input = pn.widgets.FloatInput(name="width", start=1, end=2000, step=10, value=400, width=80)
n_lobe_input = pn.widgets.FloatInput(name="lobes", start=0, end=100, step=1, value=3, width=80)
apodization_input = pn.widgets.FloatInput(name="apodization", start=0, end=100, step=1, value=0, width=80)

p1 = ComplexPlot(
    title="Waveform",
    x_axis_label="Time (ms)",
    y_axis_label="Amplitude",
    height=400)

p2 = ComplexPlot(
    title="Spectrum",
    x_axis_label="Frequency (kHz)",
    y_axis_label="Spectral Density",
    height=400)

plot_row = pn.Row(p1.figure, p2.figure, sizing_mode='stretch_width')

global_shape = [1]
global_width = 100e-6

def update_plots(event):
    global global_width, global_shape 
    width = width_input.value*1e-6
    N = int(width / 1e-6)
    if N > 1000:
        N = 1000
    dt = width / N
    pts = np.sinc(np.linspace(-n_lobe_input.value, n_lobe_input.value,N)) 
    t = np.arange(N)*dt
    t_0 = (width-dt)/2
    pts*=np.exp(-apodization_input.value*(np.linspace(-1, 1, N)**2))
    global_shape = pts
    global_width = width
    freq = fftfreq(N,dt)
    spectrum = fft(pts)
    spectrum *= np.exp(-1j*np.pi*(width+dt)*freq) # fix phase of spectrum plot due to time offset
    freq = fftshift(freq)
    spectrum = fftshift(spectrum)
    p1.update_data(t, pts)
    p2.update_data(freq, spectrum)
    pn.io.push_notebook(plot_row)

# update plot when any value is changed
width_input.param.watch(update_plots, 'value')
n_lobe_input.param.watch(update_plots, 'value')
apodization_input.param.watch(update_plots, 'value')

# manually trigger for first update
width_input.param.trigger('value')

app = pn.Row(
    pn.Column(width_input, n_lobe_input, apodization_input),
    plot_row,
    sizing_mode='stretch_width')

app

Row(sizing_mode='stretch_width')
    [0] Column
        [0] FloatInput(end=2000, name='width', start=1, step=10, value=400, width=80)
        [1] FloatInput(end=100, name='lobes', start=0, step=1, value=3, width=80)
        [2] FloatInput(end=100, name='apodization', start=0, step=1, width=80)
    [1] Row(sizing_mode='stretch_width')
        [0] Bokeh(Figure)
        [1] Bokeh(Figure)

TO DO: 
- Pulse Sequence diagrams of refocusing gradient
- Explanation of exciting a slice and that the slice profile is plotting
- Talk about refocusing gradient and why it's used 

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.1.2: Refocusing Gradient**
> TO DO: Insert a window that runs a pulse sequence with and without a refocusing gradient and plots the slice profile. Students can see how this causes the phase to be scrambled etc. Adding the refocusing gradient changes this. 
> 1. Use
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
from custom_pulse_SE_refocusing_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

input_refocusing = pn.widgets.Checkbox(name='Refocusing')

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    target_area = 0.3*32e-6
    return min(1, abs(target_area/shape_area))

# set some parameters directly
override_pars = dict(
    a_90=estimate_a_90,
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=(0,0,-0.5),
    g_read=(0,0,0.5),
    n_scans=2,
    n_samples=200,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5, 
    enable_refocusing = input_refocusing
)

# create dashboard app
exp2_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp2_app.plot1.figure.height=400
exp2_app.plot2.figure.height=400

# display layout
pn.Column(
    input_refocusing,
    exp2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Checkbox(name='Refocusing')
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

TO DO: 
- Talk about non linearity and why it matters 

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.1.3: Non Linearity of Sinc Pulse**
> TO DO: Insert a window that runs a pulse sequence and plots the slice profile. Amplitude of the RF pulse can be altered to show how the shape of the slice profile changes based on the amplitude of the pulse. 
> 1. Use
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
from custom_pulse_SE_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

amplitude_input = pn.widgets.FloatInput(name="Amplitude", start=1, end=360, step=10, value=90, width=80)

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    target_area = 0.3*32e-6
    return min(1, abs(target_area/shape_area))

# set some parameters directly
override_pars = dict(
    a_90=lambda: (amplitude_input.value/90)*estimate_a_90(),
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=(0,0,-0.5),
    g_read=(0,0,0.5),
    n_scans=2,
    n_samples=200,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
exp3_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp3_app.plot1.figure.height=400
exp3_app.plot2.figure.height=400

# display layout
pn.Column(
    amplitude_input,
    exp3_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] FloatInput(end=360, name='Amplitude', start=1, step=10, value=90, width=80)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

#### 3.1.2 SLR Pulse Design
TO DO: Explain why based on the last exercises that optimised pulses are better than sinc pulses for what we're doing. Talk about the differences between them. 


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1.2.1: SLR Pulse Calibration**
> TO DO: Insert a window that runs a pulse sequence and plots the slice profile. Amplitude of the RF pulse can be altered to show how the shape of the slice profile changes based on the amplitude of the pulse. 
> 1. Use
> -------------------------------------------------------------------------------------------------------------------------------------------------------

### 3.2 Slice Select Gradient

The thickness of the slice that is selected is also affected by the strength of the gradient. 

Fig.x illustrates how three identical RF-pulses can result in different slice thicknesses when the gradients are changed.

<center><img src="Images/slice_thickness.png" width="1400"></center>
<center><figcaption style="width: 300px;">Figure x:  </figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.2.1: ???**
> TO DO: Insert 2D RARE that is set to show just the width of the slice. Have adjustable bandwidth and gradient parameters so they can see the slice width change as the parameters are changed.
> 1. Change the...
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In many of ilumr's dashboards you can select the desired slice thickness and the bandwidth and gradient will both be automatically adjusted to allow for this thickness. This is done by... 

## 4. Controlling Slice Position
TO DO: Explain centre frequency and how this is controlled. Maybe an equation 

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.2: Locating the Hidden Text**
> TO DO: insert a standard 2D RARE dashboard where they can play around with slice selection in different axes to find the ilumr text in the ilumr phantom
> 1. Change the...
> -------------------------------------------------------------------------------------------------------------------------------------------------------

Things to add:
- Need to add a section talking about center frequency for the slice selection pulse and how this allows you to slice at different points in the sample
- Changing the direction/axis of slice selection
- Non-linearity of Bloch equations (talk about distortion in slice profile once go past a certain point)
- How the pulse calibration dashboard works 
- Calibration methods
- Tying everything to physical measurements - how does the actual gradient strength and B1 relate to physical slice thickness. How to calculate this.